# Goal

Train model

Thanks to: OBELISK, FAIMED3D, MONAI
- https://github.com/mattiaspaul/OBELISK
-  https://github.com/kbressem/faimed3d/blob/main/examples/3d_segmentation.md

# Setup parameters

In [11]:
import sys

kwargs = dict(arg.split("=") for arg in sys.argv if "=" in arg)
print(kwargs)

do_reload = "model_type" not in kwargs # True #True #True #False
if do_reload:
    %load_ext autoreload
    %autoreload 2
    
    model_type = "UNET3D" # VNET, UNET3D, UNETR, SegResNetVAE, OBELISKHYBRID
    loss_type  = "BCE_loss" # DICE_loss
    pixdim     = tuple(1.5 for _ in range(3))
    full_res   = tuple(96 for _ in range(3))
    do_flip    = True
    do_simple  = True
    do_test    = False # False

else:
    model_type = kwargs["model_type"]
    loss_type  = kwargs["loss_type"]
    pixdim     = kwargs["pixdim"]
    full_res   = kwargs["full_res"]
    do_flip    = kwargs["do_flip"]
    do_simple  = kwargs["do_simple"]
    do_test    = False

    # tuple
    pixdim   = tuple(float(pixdim) for _ in range(3))
    full_res = tuple(int(full_res) for _ in range(3))

    print(f"Model: {model_type}")
    print(f"Loss : {loss_type}")
    print(f"Pixd : {pixdim}")
    print(f"Fullres : {full_res}")
    print(f"Do flip: {do_flip}")
    print(f"Do simple: {do_simple}")

{}
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# model_type = "UNETR" # VNET, UNET3D, UNETR, SegResNetVAE, OBELISKHYBRID
# loss_type  = "BCE_loss" # DICE_loss
# pixdim     = tuple(1.5 for _ in range(3))
# full_res   = tuple(96 for _ in range(3))
# do_flip    = True

In [13]:
# import os

# try:
#     taskid = int(os.getenv('SLURM_ARRAY_TASK_ID'))
# except:
#     taskid = 0

# taskid = 1
# print(f"Taskid: {taskid}")

# if taskid == 0:
#     # MODEL & LOSS
#     model_type = "UNET3D" #"SegResNetVAE" # "UNET3D" #"OBELISKHYBRID" #"VNET" # "UNET3D" "OBELISKHYBRID"
#     loss_type  = "BCE_loss" #"log_cosh_dice_loss" #"vae_loss" #"perim_loss" #"log_cosh_dice_loss" # DICE
# elif taskid == 1:
#     # MODEL & LOSS
#     model_type = "UNET3D" #"SegResNetVAE" # "UNET3D" #"OBELISKHYBRID" #"VNET" # "UNET3D" "OBELISKHYBRID"
#     loss_type  = "log_cosh_dice_loss" #"log_cosh_dice_loss" #"vae_loss" #"perim_loss" #"log_cosh_dice_loss" # DICE
# else:
#     print("Taskid {taskid} not recognized.")
    
# print(f"Model: {model_type}")
# print(f"Loss : {loss_type}")

In [14]:
# MODEL & LOSS
# model_type = "UNET3D" #"SegResNetVAE" # "UNET3D" #"OBELISKHYBRID" #"VNET" # "UNET3D" "OBELISKHYBRID"
# loss_type  = "BCE_loss" #"log_cosh_dice_loss" #"vae_loss" #"perim_loss" #"log_cosh_dice_loss" # DICE

# DATALOADER PARAMS
bs          = 2
nepochs     = 60
num_workers = 2

# full_res      = (96,96,96)
#full_res      = (144,144,144)

In [15]:
from helpers.model_loss_choices import get_model, get_loss

model   = get_model(model_type, full_res)
loss_fn = get_loss(loss_type) 

In [16]:
# model

In [17]:
import os, shutil

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference


from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import UNETR

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)


import torch

print_config()

MONAI version: 0.7.dev2131
Numpy version: 1.19.5
Pytorch version: 1.7.1+cu101
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 57467c75bff90e6c9da74461f7da3a828a39626b

Optional dependencies:
Pytorch Ignite version: 0.4.5
Nibabel version: 3.2.1
scikit-image version: 0.17.2
Pillow version: 8.3.1
Tensorboard version: 2.5.0
gdown version: 3.13.0
TorchVision version: 0.8.2+cu101
tqdm version: 4.62.0
lmdb version: 1.2.1
psutil version: 5.8.0
pandas version: 1.1.5
einops version: 0.3.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



# Items as dict 

In [20]:
from helpers.items_constants import *
train_itemsd = getd(train_items[:20])
val_itemsd   = getd(valid_items[:20])
print(f"train: {len(train_itemsd)}, val: {len(val_itemsd)}")

train: 20, val: 20


# Transforms

In [21]:
from helpers.transforms import UndoDict, get_train_valid_transforms, get_train_valid_transforms_simple, monai_tfms2str

if do_simple:
    train_tfms, val_tfms = get_train_valid_transforms_simple(pixdim=pixdim, full_res=full_res)
else:
    train_tfms, val_tfms = get_train_valid_transforms(pixdim=pixdim, full_res=full_res, do_flip=do_flip)

In [22]:
# Fastai + distributed training
from fastai              import *
from fastai.torch_basics import *
from fastai.basics       import *
from fastai.distributed  import *
from fastai.callback.all import SaveModelCallback, CSVLogger

In [23]:
# tls, dls, cuda
train_dl = TfmdDL(train_itemsd, after_item=train_tfms, after_batch=[], bs=bs)
val_dl   = TfmdDL(val_itemsd,   after_item=val_tfms,   after_batch=[], bs=bs)

# tls, dls, cuda
if do_test:
    train_dl = TfmdDL(train_itemsd[:10], after_item=train_tfms, after_batch=[], bs=bs)
    val_dl   = TfmdDL(val_itemsd[:10],   after_item=val_tfms,   after_batch=[], bs=bs)
    nepochs = 2

In [24]:
dls = DataLoaders(train_dl, val_dl)
dls = dls.cuda()

In [25]:
# # Viz

# from helpers.viz        import viz_axis, viz_compare_inputs, viz_compare_outputs
# from helpers.preprocess import mask2bbox
# from helpers.general    import lrange

# train_dl = TfmdDL(train_itemsd, after_item=train_tfms, after_batch=[], bs=3)

# b = train_dl.one_batch()

# xb,yb = b

# print("shape xb yb", xb.shape, yb.shape)

# i = 2
# mr = np.array(xb[i].squeeze().cpu())
# mk = np.array(yb[i].squeeze().cpu())

# bbox = mask2bbox(mk)

# viz_axis(np_arr = mr, \
#     bin_mask_arr   = mk,     color1 = "yellow",  alpha1=0.3, \
#     slices=lrange(*bbox[0:2]), fixed_axis=0, \
#     axis_fn = np.rot90, \
#     title   = "Axis 0", \

#     np_arr_b = mr, \
#     bin_mask_arr_b   = mk,     color1_b = "yellow",  alpha1_b=0.3, \
#     slices_b = lrange(*bbox[2:4]), fixed_axis_b=1, \
#     title_b  = "Axis 1", \

#     np_arr_c = mr, \
#     bin_mask_arr_c   = mk,     color1_c = "yellow",  alpha1_c=0.3, \
#     slices_c = lrange(*bbox[4:6]), fixed_axis_c=2, \
#     title_c = "Axis 2", \
  
# ncols = 5, hspace=0.3, fig_mult=2)

In [26]:
# Utilities
import os, sys, gc, time, pickle
from pathlib import Path

# Fastai + distributed training
from fastai              import *
from fastai.torch_basics import *
from fastai.basics       import *
from fastai.distributed  import *
from fastai.callback.all import SaveModelCallback, CSVLogger

In [27]:
# length  = len(items)
# indices = np.arange(length)
# np.random.shuffle(indices)
# #rank0_first(lambda: np.random.shuffle(indices))

# test_split   = int(test_frac  * length)
# valid_split  = int(valid_frac * length) + test_split

# test_idxs    = indices[:test_split] 
# valid_idxs   = indices[test_split:valid_split]
# train_idxs   = indices[valid_split:]

# train_items = [items[i] for i in train_idxs]
# valid_items = [items[i] for i in valid_idxs]
# test_items  = [items[i] for i in test_idxs]

# # print
# print(f"Total  {len(items)} items in dataset.")
# print(f"Train: {len(train_items)} items.")
# print(f"Valid: {len(valid_items)} items.")
# print(f"Test:  {len(test_items)} items.")

In [28]:
# # save test set indices
# with open(f"{data_src}/saved_dset_metadata/split_train_valid_test.pkl", 'wb') as f:
#     pickle.dump([train_idxs, valid_idxs, test_idxs, train_items, valid_items, test_items], f)

In [29]:
# test get one batch
# time_one_batch(dls)

In [30]:
# Save test idxs + model + runs
from helpers.time       import time_one_batch, get_time_id

# file name
model_time = rank0_first(lambda:get_time_id()) # 'Mon Oct 18 13:35:29 2010'
model_name = f"model_{model_type}_loss_{loss_type}_full_res_{full_res[0]}_pixdim_{pixdim[0]}_do_simple_{do_simple}_do_flip_{do_flip}_bs_{bs}_epochs_{nepochs}_time_{model_time}"
print(f"Model name: {model_name}")

Model name: model_UNET3D_loss_BCE_loss_full_res_96_pixdim_1.5_do_simple_True_do_flip_True_bs_2_epochs_60_time_1627953225_Mon_Aug_02_2021_hr_21_min_13


# Save

In [31]:
# make dir
model_src = f"{run_src}/{model_name}"
fig_src = f"{model_src}/figs"
Path(fig_src).mkdir(parents=True, exist_ok=True)

In [32]:
if not do_test:
    old_stdout = sys.stdout
    log_file = open(f"{model_name}.log","w")
    print(kwargs)
    print(f"Model name: {model_name}")
    print(f"train: {len(train_itemsd)}, val: {len(val_itemsd)}")

{}
Model name: model_UNET3D_loss_BCE_loss_full_res_96_pixdim_1.5_do_simple_True_do_flip_True_bs_2_epochs_60_time_1627953225_Mon_Aug_02_2021_hr_21_min_13
train: 20, val: 20


In [33]:
# save test set indices
# with open(f"{model_src}/test_items.pkl", 'wb') as f:
#     rank0_first(lambda:pickle.dump(list(test_items), f))

# save data augs
with open(f"{model_src}/data_augs.txt", 'w') as f:
    def print_data_augs():
        print("Train Tfms: ", file=f); print(monai_tfms2str(train_tfms), file=f)
        print("Val   Tfms: ", file=f); print(monai_tfms2str(val_tfms), file=f)
        
    rank0_first(print_data_augs)

#     s = "\n".join([str(tfm) for tfm in train_tfms.transforms] + [f"Pixdim: 1.5"])
#     rank0_first(lambda: print(s, file=f))
    
# with open(f"{model_src}/{model_name}_test_items.pkl", 'wb') as f:
#     rank0_first(lambda:pickle.dump(list(test_items), f))

# Callbacks

In [34]:
cbs = [
    rank0_first(lambda: Recorder(train_metrics=False)),
    rank0_first(lambda: SaveModelCallback(monitor='dice_score', with_opt=True)), 
    rank0_first(lambda: CSVLogger(fname=f"{fig_src}/history.csv"))
]

#no more metrics
# rank0_first(lambda: SaveModelCallback(monitor='valid_dice_score', with_opt=True)), 


# class PerimLossMidway(Callback):
#     def before_epoch(self): 
#         if self.epoch == self.n_epoch//2:
#             self.learn.loss_func = perim_loss
#             print("Changed ", self.learn.loss_func, "at epoch ", self.n_epoch//2)
            
# cbs = [SaveModelCallback(monitor='dice_score', with_opt=True), CSVLogger(fname=f"{fig_src}/history.csv")]

# if loss_type == "perim_loss":
#     print("added PerimLoss callback")
#     cbs.append(PerimLossMidway())

# Learner

In [35]:
# clear cache
from helpers.general import print_hardware_stats

gc.collect()
torch.cuda.empty_cache()
rank0_first(lambda: print_hardware_stats())

#GPU = 1, #CPU = 40
GPU Tesla V100-SXM2-16GB RAM Free: 16157MB | Used: 3MB | Util   0% | Total 16160MB


In [36]:
from helpers.losses import dice_score

In [86]:
learn = rank0_first(lambda:
            Learner(dls   = dls, \
                model     = model, \
                loss_func = loss_fn, \
                metrics   = dice_score, \
                model_dir = model_src, \
                cbs       = cbs)
        )

# cbs TensorBoardCallback(Path(run_src)/model_name, trace_model=True)
# GPU
learn.model = rank0_first(lambda:learn.model.cuda())

In [87]:
# # check
# print("Check")
# b = dls.one_batch()
# xb,yb = b #b["image"], b["label"]
# print(f"Batch: {len(b)}. xb: {xb.shape}, yb: {yb.shape}")
# predb = learn.model(xb)
# print(f"Pred batch: {predb.shape}")
# loss = loss_fn(predb, yb)
# print(f"Loss: {loss}")

In [88]:
# print(predb[0].shape, predb[1])

# LR Finder

In [89]:
# print("PRE learn.fit one cycle")
# with learn.distrib_ctx():
#     learn.fit_one_cycle(2, 3e-3, wd = 1e-4)

In [90]:
# SuggestedLRs(valley=tensor(0.0229))
# learn.lr_find()

In [91]:
print("PRE learn.fit one cycle")

with learn.distrib_ctx():
    learn.fit_one_cycle(nepochs, 3e-3, wd = 1e-4)
    

PRE learn.fit one cycle


Better model found at epoch 0 with valid_dice_score value: 0.003745168913155794.
Better model found at epoch 9 with valid_dice_score value: 0.3832036852836609.
Better model found at epoch 10 with valid_dice_score value: 0.46944355964660645.
Better model found at epoch 12 with valid_dice_score value: 0.5245916247367859.
Better model found at epoch 33 with valid_dice_score value: 0.5438696146011353.
Better model found at epoch 36 with valid_dice_score value: 0.5579066276550293.
Better model found at epoch 41 with valid_dice_score value: 0.5682535171508789.
Better model found at epoch 42 with valid_dice_score value: 0.5771719217300415.
Better model found at epoch 46 with valid_dice_score value: 0.6057624220848083.
Better model found at epoch 51 with valid_dice_score value: 0.6086040735244751.


In [92]:
#     if loss_type == "perim_loss":
#         learn.loss_func = perim_loss
#         print("switched loss at epoch ", nepochs//2)
    
#     learn.fit_one_cycle(nepochs//2, 3e-3, wd = 1e-4)

In [93]:
# learn.recorder.plot_loss()

In [94]:
def save_plot_loss(self, skip_start=5, with_valid=True):
        plt.plot(list(range(skip_start, len(self.losses))), self.losses[skip_start:], label='train')
        if with_valid:
            idx = (np.array(self.iters)<skip_start).sum()
            plt.plot(self.iters[idx:], L(self.values[idx:]).itemgot(1), label='valid')
            plt.legend()
        plt.savefig(f'{fig_src}/loss.png', bbox_inches='tight')
        plt.close()

In [95]:
# save_plot_loss(learn.recorder)

In [96]:
# self = learn.recorder
# nrows=None 
# ncols=None
# figsize=None,
# metrics = np.stack(self.values)
# names = self.metric_names[1:-1]
# n = len(names) - 1
# if nrows is None and ncols is None:
#     nrows = int(math.sqrt(n))
#     ncols = int(np.ceil(n / nrows))
# elif nrows is None: nrows = int(np.ceil(n / ncols))
# elif ncols is None: ncols = int(np.ceil(n / nrows))
# figsize = (ncols * 6, nrows * 4)
# fig, axs = subplots(nrows, ncols, figsize=figsize)
# axs = [ax if i < n else ax.set_axis_off() for i, ax in enumerate(axs.flatten())][:n]
# for i, (name, ax) in enumerate(zip(names, [axs[0]] + axs)):
#     ax.plot(metrics[:, i], color='#1f77b4' if i == 0 else '#ff7f0e', label='valid' if i > 0 else 'train')
#     ax.set_title(name if i > 1 else 'losses')
#     ax.legend(loc='best')

In [97]:
@delegates(subplots)
def save_plot_metrics(self: Recorder, nrows=None, ncols=None, figsize=None, **kwargs):
    metrics = np.stack(self.values)
    names = self.metric_names[1:-1]
    n = len(names) - 1
    if nrows is None and ncols is None:
        nrows = int(math.sqrt(n))
        ncols = int(np.ceil(n / nrows))
    elif nrows is None: nrows = int(np.ceil(n / ncols))
    elif ncols is None: ncols = int(np.ceil(n / nrows))
    figsize = figsize or (ncols * 6, nrows * 4)
    fig, axs = subplots(nrows, ncols, figsize=figsize, **kwargs)
    axs = [ax if i < n else ax.set_axis_off() for i, ax in enumerate(axs.flatten())][:n]
    for i, (name, ax) in enumerate(zip(names, [axs[0]] + axs)):
        ax.plot(metrics[:, i], color='#1f77b4' if i == 0 else '#ff7f0e', label='valid' if i > 0 else 'train')
        ax.set_title(name if i > 1 else 'losses')
        ax.legend(loc='best')
    #plt.show()
    plt.savefig(f'{fig_src}/metrics.png', bbox_inches='tight')
    plt.close()

In [98]:
# if do_test:
#     self = learn.recorder
#     nrows =None; ncols=None; figsize=None

#     metrics = np.stack(self.values)
#     # 'train_loss', 'train_dice_score', 'valid_loss', 'valid_dice_score'
#     names = self.metric_names[1:-1]
#     print("Metric names: ", names)

#     names_train = [n for n in names if n.startswith("train")]
#     n = len(names_train)
#     if nrows is None and ncols is None:
#         nrows = int(math.sqrt(n))
#         ncols = int(np.ceil(n / nrows))
#     elif nrows is None: nrows = int(np.ceil(n / ncols))
#     elif ncols is None: ncols = int(np.ceil(n / nrows))
#     figsize = (ncols * 6, nrows * 4)
#     fig, axs = subplots(nrows, ncols, figsize=figsize)
#     for i, ax in enumerate(axs):
#         name = names_train[i]
#         n = name[name.index("_")+1:]
#         valid_name = f"valid_{n}"
#         valid_idx = names.index(valid_name)
#         print(i, valid_idx, name, valid_name)
#         ax.plot(metrics[:, i], color='#1f77b4',  label='train')
#         ax.plot(metrics[:, valid_idx], color = '#ff7f0e', label='valid')
#         ax.set_title(n)
#         ax.legend(loc='best')

In [106]:
# @delegates(subplots)
# def save_plot_metrics(self: Recorder, nrows=None, ncols=None, figsize=None, **kwargs):
#     metrics = np.stack(self.values)
#     # 'train_loss', 'train_dice_score', 'valid_loss', 'valid_dice_score'
#     names = self.metric_names[1:-1]
#     print("Metric names: ", names)
    
#     names_train = [n for n in names if n.startswith("train")]
#     n = len(names_train)
#     if nrows is None and ncols is None:
#         nrows = int(math.sqrt(n))
#         ncols = int(np.ceil(n / nrows))
#     elif nrows is None: nrows = int(np.ceil(n / ncols))
#     elif ncols is None: ncols = int(np.ceil(n / nrows))
#     figsize = (ncols * 6, nrows * 4)
#     fig, axs = subplots(nrows, ncols, figsize=figsize)
#     for i, ax in enumerate(axs):
#         name = names_train[i]
#         n = name[name.index("_")+1:]
#         valid_name = f"valid_{n}"
#         valid_idx = names.index(valid_name)
#         print(i, valid_idx, name, valid_name)
#         ax.plot(metrics[:, i], color='#1f77b4',  label='train')
#         ax.plot(metrics[:, valid_idx], color = '#ff7f0e', label='valid')
#         ax.set_title(n)
#         ax.legend(loc='best')
#     #plt.show()
#     plt.savefig(f'{fig_src}/metrics.png', bbox_inches='tight')
#     plt.close()

In [107]:
save_plot_metrics(learn.recorder)

Metric names:  ['train_loss', 'train_dice_score', 'valid_loss', 'valid_dice_score']
0 2 train_loss valid_loss
1 3 train_dice_score valid_dice_score


In [108]:
print(model_name)

model_UNETR_loss_BCE_loss_full_res_96_pixdim_1.5_do_flip_True_bs_1_epochs_60_time_1627918222_Mon_Aug_02_2021_hr_11_min_30


In [109]:
if not do_test:
    sys.stdout = old_stdout
    log_file.close()

# Old

In [103]:
# batch_tfms = [
#     # normalize mean/std of foreground pixels
#     ZScale(),
#     # affine + flips
#     RandomAffine(p=0.5, degrees=35, translate=0.1, scale=0.1),
#     RandDihedral(p=0.5),
#     # lighting
#     RandBright(p=0.5),
#     RandContrast(p=0.5),
#     # noise for generalizability
#     GNoise(p=0.5),
#     GBlur(p=0.5),
#     # add channel dim
#     AddChannel()

# UMich 
# code src: "/home/labcomputer/Desktop/Rachel"
# data src: "../../../../..//media/labcomputer/e33f6fe0-5ede-4be4-b1f2-5168b7903c7a/home/rachel/"

# ]

# Test

In [104]:
# print("Test")
# xb, yb = dls.one_batch()
# xb, yb = xb.cpu(), yb.cpu()

# pb = model.cpu()(xb)
# print(xb.shape, pb.shape)
# print(f"logcosh dice loss {log_cosh_dice_loss(pb,yb)}")

In [ ]:
# # test:

# #dls.device = "cpu"

# start = time.time()

# x,y = dls.one_batch()
# #x,y = to_cpu(x), to_cpu(y)

# pred = learn.model(x)
# loss = learn.loss_func(pred, y)

# elapsed = time.time() - start

# print(f"Elapsed: {elapsed} s")
# print("Batch: x,y")
# print(type(x), x.shape, x.dtype, "\n", type(y), y.shape, y.dtype)

# print("Pred shape")
# print(type(pred), pred.shape, pred.dtype)

# print("Loss")
# print(loss)
# print(learn.loss_func)